In [2]:
import parse_inventory as pi
import parse_dealership
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import sqlite3
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [59]:
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
}

URL = 'https://www.streamautooutlet.com/vehicle-details/used-2020-jeep-grand-cherokee-limited-x-1C4RJFBG3LC201198'
response = requests.get(URL, headers = headers)
vehicle_detail = BeautifulSoup(response.text, "html.parser")


In [60]:
# https://www.streamautooutlet.com/vehicle-details/used-2020-jeep-grand-cherokee-limited-x-1C4RJFBG3LC201198

print(soup.prettify())

<!DOCTYPE html>
<!--[if IE 9]><html class="lt-ie10" lang="en" > <![endif]-->
<html class="no-js vehicle-detailsused-2020-jeep-grand-cherokee-limited-x-1C4RJFBG3LC201198">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1, user-scalable=yes, maximum-scale=1, height=device-height" name="viewport">
   <script type="application/ld+json">
    {
    "@context": "https://schema.org",
    "@type": "AutoDealer",
    "@id": "1973",
    "name": "Stream Auto Outlet",
    "url": "https://www.streamautooutlet.com",
    "telephone": "(516) 490-5459",
    "logo": "https://s3.amazonaws.com/fzautomotive/dealers/5f2d7ee0df268.png",
    "image": "https://s3.amazonaws.com/fzautomotive/dealers/5f2d7ee0df268.png",
    "priceRange": "$$$$$",
    "title": null,
    "description": null,
    "hasMap": "https://www.google.com/maps/dir/324 W Merrick Rd, Valley Stream, NY 11580",
    "address": {
        "@type": 

In [7]:
main = soup.findAll('script')

In [14]:
main[0]

<script type="application/ld+json"> 
                    {
    "@context": "https://schema.org",
    "@type": "AutoDealer",
    "@id": "1973",
    "name": "Stream Auto Outlet",
    "url": "https://www.streamautooutlet.com",
    "telephone": "(516) 490-5459",
    "logo": "https://s3.amazonaws.com/fzautomotive/dealers/5f2d7ee0df268.png",
    "image": "https://s3.amazonaws.com/fzautomotive/dealers/5f2d7ee0df268.png",
    "priceRange": "$$$$$",
    "title": null,
    "description": null,
    "hasMap": "https://www.google.com/maps/dir/324 W Merrick Rd, Valley Stream, NY 11580",
    "address": {
        "@type": "PostalAddress",
        "streetAddress": "324 W Merrick Rd",
        "addressLocality": "Valley Stream",
        "addressRegion": "NY",
        "postalCode": "11580",
        "addressCountry": {
            "@type": "Country",
            "name": "America"
        }
    },
    "geo": {
        "@type": "GeoCoordinates",
        "latitude": "",
        "longitude": null
    },
    "O

In [23]:
data = ''
for el in main:
    if "fzDataLayer['vehicle']" in el.get_text():
        data = el.get_text()
        print(data)


		fzDataLayer['vehicle'] = {
			"type": "Used",
			"year": "2020",
			"make": "Jeep",
			"model": "Grand Cherokee",
			"trim": "Limited X",
			"vin": "1C4RJFBG3LC201198",
			"stock": "2724",
			"modelnumber": "WKJP74",
			"extcolor": "Black",
			"msrp": "0",
			"sellingprice": "38990",
			"internetprice": "0",
            "invoice": "0",
            
			"body": "SUV"
		};



In [22]:
import re

In [50]:
x = re.search('"trim": (.)+,', data)
print(x)

<re.Match object; span=(120, 140), match='"trim": "Limited X",'>


In [55]:
trim = x.group(0)
trim = trim.replace('"trim": ', '').replace('"', '').replace(',', '')

In [56]:
print(trim)

Limited X


## Data

### Title

In [63]:
title = pi.clean_text_data(pi.parse_subsection_all(vehicle_detail, 'div', 'span', 'columns vehicle-title'))
print(title)

['2020 Jeep Grand Cherokee Limited X SUV']


### Year, Make, Model, Trim

In [70]:
data = ''
for el in main:
    if "fzDataLayer['vehicle']" in el.get_text():
        data = el.get_text()


print(data)


		fzDataLayer['vehicle'] = {
			"type": "Used",
			"year": "2020",
			"make": "Jeep",
			"model": "Grand Cherokee",
			"trim": "Limited X",
			"vin": "1C4RJFBG3LC201198",
			"stock": "2724",
			"modelnumber": "WKJP74",
			"extcolor": "Black",
			"msrp": "0",
			"sellingprice": "38990",
			"internetprice": "0",
            "invoice": "0",
            
			"body": "SUV"
		};



In [103]:
year = re.search('"year": (.)+,', data)
year = int(year.group(0).replace('"year": ', '').replace('"', '').replace(',', ''))
print(year)

2020


In [78]:
make = re.search('"make": (.)+,', data)
make = make.group(0).replace('"make": ', '').replace('"', '').replace(',', '')
print(make)

Jeep


In [74]:
model = re.search('"model": (.)+,', data)
model = model.group(0).replace('"model": ', '').replace('"', '').replace(',', '')
print(model)

Grand Cherokee


In [75]:
trim = re.search('"trim": (.)+,', data)
trim = trim.group(0).replace('"trim": ', '').replace('"', '').replace(',', '')
print(trim)

Limited X


### Other Data

In [93]:
# vdp-vehicle-details
add_data = pi.get_row_subsection_data(vehicle_detail, ('div', 'columns show-for-small-only'), ('ul', 'no-bullet'))
print(add_data)

[[<ul class="no-bullet">
<li><span>Body Style:</span> SUV</li><li><span>Model Code:</span> WKJP74</li><li><span>Engine:</span> Regular Unleaded V-6 3.6 L/220</li><li><span>Transmission:</span> Automatic</li><li><span>Drive Type:</span> 4WD</li><li><span>Ext. Color:</span> Diamond Black Crystal Pearlcoat</li><li><span>Int. Color:</span> Black</li><li><span>Mileage:</span> 29,680</li><li><span>Stock #:</span> 2724</li><li><span>VIN</span> 1C4RJFBG3LC201198</li> </ul>]]


In [94]:
vehicle_data = []
for section in add_data:
  sub_data = []
  for el in section:
      el_data = [x.get_text() for x in el.find_all('li')]
      sub_data += el_data
  vehicle_data.append(sub_data)

In [106]:
print(vehicle_data[0])

['Body Style: SUV', 'Model Code: WKJP74', 'Engine: Regular Unleaded V-6 3.6 L/220', 'Transmission: Automatic', 'Drive Type: 4WD', 'Ext. Color: Diamond Black Crystal Pearlcoat', 'Int. Color: Black', 'Mileage: 29,680', 'Stock #: 2724', 'VIN 1C4RJFBG3LC201198']


In [104]:
exterior_color = None
interior_color = None
transmission = None
engine = None
drivetrain = None
vin = None
vehicle_type = None
mileage = None

for col in vehicle_data:
  if 'Ext. Color' in col:
    cleaned_str = col.replace('Ext. Color: ', '')
    exterior_color = cleaned_str
  elif 'Int. Color' in col:
    cleaned_str = col.replace('Int. Color: ', '')
    interior_color = cleaned_str
  elif 'Transmission' in col:
    cleaned_str = col.replace('Transmission: ', '')
    transmission = cleaned_str
  elif 'Mileage' in col:
    cleaned_str = col.replace('Mileage: ', '').replace(',', '')
    mileage = cleaned_str
  elif 'Drivetrain' in col:
    cleaned_str = col.replace('Drivetrain: ', '')
    drivetrain = cleaned_str
  elif 'Engine' in col:
    cleaned_str = col.replace('Engine: ', '')
    engine = cleaned_str
  elif 'VIN' in col:
    cleaned_str = col.split(' ')[1].strip()
    vin = cleaned_str
  elif 'Body Style' in col:
    cleaned_str = col.replace('Body Style: ', '')
    vehicle_type = cleaned_str

In [105]:
print(mileage)

None


In [97]:
cars_dict = {
    'title': [],
    'year': [],
    'make': [],
    'model': [],
    'trim': [],
    'model_trim': [],
    'price': [],
    'mileage': [],
    'exterior_color': [],
    'interior_color': [],
    'transmission': [],
    'engine': [],
    'drivetrain': [],
    'vin': [],
}


for i in range(len(vehicle_data)):
    row = {}
    for col in vehicle_data[i]:
      if 'Ext. Color' in col:
        cleaned_str = col.replace('Ext. Color: ', '')
        row['exterior_color'] = cleaned_str
      elif 'Int. Color' in col:
        cleaned_str = col.replace('Int. Color: ', '')
        row['interior_color'] = cleaned_str
      elif 'Transmission' in col:
        cleaned_str = col.replace('Transmission: ', '')
        row['transmission'] = cleaned_str
      elif 'Mileage' in col:
        cleaned_str = col.replace('Mileage: ', '').replace(',', '')
        row['mileage'] = cleaned_str
      elif 'Drivetrain' in col:
        cleaned_str = col.replace('Drivetrain: ', '')
        row['drivetrain'] = cleaned_str
      elif 'Engine' in col:
        cleaned_str = col.replace('Engine: ', '')
        row['engine'] = cleaned_str
      elif 'VIN' in col:
        cleaned_str = col.split(' ')[1].strip()
        row['vin'] = cleaned_str
    
for key in ['exterior_color', 'interior_color', 'drivetrain', 'transmission', 'mileage', 'engine', 'vin']:
  if key in row:
      vehicle_misc_info[key].append(row[key])
  else:
      vehicle_misc_info[key].append(None)

In [98]:
vehicle_misc_info

{'exterior_color': ['Diamond Black Crystal Pearlcoat'],
 'interior_color': ['Black'],
 'transmission': ['Automatic'],
 'engine': ['Regular Unleaded V-6 3.6 L/220'],
 'drivetrain': [None],
 'vin': ['1C4RJFBG3LC201198'],
 'mileage': ['29680']}

In [101]:
pi.convert_to_numeric_type(
    pi.parse_subsection(vehicle_detail, 'div', 'span', 'row', 'price-value right', 'get_text')
)

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 38990,
 nan,
 38990,
 nan,
 nan,
 nan,
 nan,
 38990,
 38990,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]